In [7]:
import requests
from google.cloud import bigquery
import pandas as pd
import datetime
from dateutil.parser import parse
import os
from datetime import datetime, timedelta

# Creamos los permisos con la llave para ingresar a Google 
# activamos el servicio
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../notebooks/Clave_Google/henry-sismo-b50b99db8b10.json"

project_id = 'your-project-id'
client = bigquery.Client(project=project_id)

In [9]:
# Creamos una tabla con los datos de la api en google cloud

# Obtener los datos de la API de USGS
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
parameters = {
    'format': 'geojson',
    'starttime': '1990-01-01',
    'endtime': '2023-06-28',
    'minmagnitude': '2.5',
    'minlatitude': '-5.266',
    'maxlatitude': '15.708',
    'minlongitude': '276.328',
    'maxlongitude': '293.906',
    'limit': 20000
}
response = requests.get(url, params=parameters)
data = response.json()

# Configurar el proyecto, dataset y tabla
project_id = 'henry-sismo'
dataset_id = 'Sismos'
table_id = 'Colombia2'

# Crear la tabla en BigQuery
client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

schema = [
    bigquery.SchemaField('id', 'STRING'),
    bigquery.SchemaField('magnitude', 'FLOAT'),
    bigquery.SchemaField('place', 'STRING'),
    bigquery.SchemaField('time', 'TIMESTAMP'),
    bigquery.SchemaField('url', 'STRING')
    # Agrega más campos según los datos que desees almacenar
]

table = bigquery.Table(table_ref, schema=schema)
client.create_table(table)

# Insertar los datos en la tabla
rows = []
for feature in data['features']:
    properties = feature['properties']
    time_str = properties['time']
    time = pd.to_datetime(time_str, unit= "ms") #parse(time_str)
    row = (
        properties['ids'],
        properties['mag'],
        properties['place'],
        time,
        properties['url']
        # Ajusta los índices de las coordenadas si es necesario
    )
    rows.append(row)

client.insert_rows(table, rows)

[]

In [ ]:
# Actualizamos la data de google query 

import requests
from google.cloud import bigquery
from datetime import datetime, timedelta


# Configurar el proyecto, dataset y tabla
project_id = 'henry-sismo'
dataset_id = 'Sismos'
table_id = 'Colombia2'

# Crear el cliente de BigQuery
client = bigquery.Client(project=project_id)

# Obtener la fecha más reciente en la tabla "Colombia"
latest_date_query = f"SELECT MAX(time) FROM `{project_id}.{dataset_id}.{table_id}`"
query_job = client.query(latest_date_query)
latest_date_result = query_job.result()

# Obtener el valor de la fecha más reciente
for row in latest_date_result:
    latest_date = row[0]

# Verificar si se obtuvo una fecha válida
if latest_date is not None:
    latest_date = latest_date.date()
else:
    # Si no hay fecha más reciente, establecer una fecha inicial
    latest_date = datetime(1990, 1, 1).date()

# Obtener la fecha actual
current_date = datetime.now().date()

# Verificar si se necesita actualizar la tabla
if current_date > latest_date:
    # Calcular la fecha de inicio y fin para la API
    start_date = latest_date + timedelta(days=1)
    end_date = current_date

    # Obtener los datos de la API de USGS
    url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
    parameters = {
        'format': 'geojson',
        'starttime': start_date.strftime('%Y-%m-%d'),
        'endtime': end_date.strftime('%Y-%m-%d'),
        'minmagnitude': '2.5',
        'minlatitude': '-5.266',
        'maxlatitude': '15.708',
        'minlongitude': '276.328',
        'maxlongitude': '293.906',
        'limit': 20000
    }
    response = requests.get(url, params=parameters)
    data = response.json()

    # Insertar los datos en la tabla
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    rows = []
    for feature in data['features']:
        properties = feature['properties']
        time_str = properties['time']
        time = pd.to_datetime(time_str, unit="ms")
        row = (
            properties['ids'],
            properties['mag'],
            properties['place'],
            time,
            properties['url']
        )
        rows.append(row)

    client.insert_rows(table, rows)

### OJO: en el archivo de requerimientos.txt de cloud functions se completa con 
google-cloud-bigquery
requests

como puedo hacer esta misma tarea, pero en google cloud

Para realizar esta tarea en Google Cloud, puedes utilizar Cloud Functions y Cloud Scheduler. Cloud Functions te permite ejecutar código de forma escalable y sin preocuparte por la infraestructura subyacente, mientras que Cloud Scheduler te permite programar la ejecución de funciones en un horario específico.

Aquí tienes los pasos a seguir para configurar esta tarea en Google Cloud:

Crea una función en Cloud Functions:

Ve a la página de Cloud Functions en la Consola de Google Cloud.
Haz clic en "Crear función".
Asigna un nombre a la función, por ejemplo, "actualizar_tabla_sismos".
Selecciona el entorno de ejecución "Python".
Copia y pega el código que te proporcioné anteriormente en el editor de código.
Asegúrate de que el código incluya las importaciones necesarias y la configuración de autenticación para BigQuery.
Define la función llamada actualizar_tabla_sismos(request), que es el punto de entrada para la función.
Guarda los cambios y crea la función.
Crea un trabajo en Cloud Scheduler:

Ve a la página de Cloud Scheduler en la Consola de Google Cloud.
Haz clic en "Crear trabajo".
Asigna un nombre al trabajo, por ejemplo, "actualizar_tabla_sismos_job".
Configura el intervalo para que se ejecute diariamente a las 13:00 en la zona horaria de Colombia. Puedes usar la expresión 0 13 * * * en el campo "Frecuencia".
En "Tipo de destino", selecciona "Cloud Pub/Sub".
Crea o selecciona un tema de Pub/Sub existente.
Haz clic en "Crear".
Configura la función para ser invocada por Cloud Scheduler:

Ve a la página de Cloud Functions en la Consola de Google Cloud.
Haz clic en el nombre de la función que creaste anteriormente.
En la pestaña "Disparadores", haz clic en "Agregar disparador".
Selecciona "Cloud Pub/Sub" como tipo de disparador.
Elige el tema de Pub/Sub que configuraste en el paso anterior.
Guarda los cambios.
Ahora, Cloud Scheduler ejecutará tu función automáticamente todos los días a las 13:00 en la zona horaria de Colombia. El código de la función verificará si es necesario actualizar la tabla "Colombia" en BigQuery y realizará la actualización si es necesario.

Asegúrate de configurar correctamente las credenciales y los permisos necesarios para acceder a BigQuery y la API de USGS en tu entorno de ejecución de Google Cloud Functions.

Para aplicar un Cloud Scheduler existente a una Cloud Function en Google Cloud Platform, sigue estos pasos:

Asegúrate de que ya tienes tanto la Cloud Function como el Cloud Scheduler creados en tu proyecto de Google Cloud Platform.

Abre la consola de Cloud Console y selecciona tu proyecto.

Navega hasta la página de la función que deseas asociar con el Cloud Scheduler. Puedes encontrar tus funciones en la sección "Cloud Functions" en el menú de servicios.

Haz clic en la función que deseas editar para abrir su configuración.

En la página de configuración de la función, busca la sección "Triggers" (Desencadenadores) y haz clic en "Add trigger" (Agregar desencadenador).

En el menú desplegable de tipos de desencadenadores, selecciona "Cloud Pub/Sub".

Configura el desencadenador de Pub/Sub con los detalles necesarios. En el campo "Topic", elige el tema de Pub/Sub asociado al Cloud Scheduler que deseas usar. También puedes especificar un identificador único para el desencadenador si lo deseas.

Haz clic en "Guardar" para guardar los cambios en la configuración de la función.

Ahora, cuando el Cloud Scheduler active el Cloud Pub/Sub, se desencadenará la función asociada y se ejecutará según lo programado.

Recuerda que debes configurar adecuadamente el Cloud Scheduler y el Cloud Pub/Sub para que se ejecuten en los horarios y las condiciones deseadas. También asegúrate de que la función se haya implementado correctamente y esté lista para manejar las solicitudes cuando se active a través del Cloud Scheduler.

In [ ]:
# Esta fue la función que se implemento e google cloud functions 

import functions_framework
import requests
from google.cloud import bigquery
from datetime import datetime, timedelta

@functions_framework.http
def hello_http(request):
    # Configurar el proyecto, dataset y tabla
    project_id = 'henry-sismo'
    dataset_id = 'Sismos'
    table_id = 'Colombia'

    # Crear el cliente de BigQuery
    client = bigquery.Client(project=project_id)

    # Obtener la fecha más reciente en la tabla "Colombia"
    latest_date_query = f"SELECT MAX(time) FROM `{project_id}.{dataset_id}.{table_id}`"
    query_job = client.query(latest_date_query)
    latest_date_result = query_job.result()

    # Obtener el valor de la fecha más reciente
    for row in latest_date_result:
        latest_date = row[0]

    # Verificar si se obtuvo una fecha válida
    if latest_date is not None:
        latest_date = latest_date.date()
    else:
        # Si no hay fecha más reciente, establecer una fecha inicial
        latest_date = datetime(1990, 1, 1).date()

    # Obtener la fecha actual
    current_date = datetime.now().date()

    # Verificar si se necesita actualizar la tabla
    if current_date > latest_date:
        # Calcular la fecha de inicio y fin para la API
        start_date = latest_date + timedelta(days=1)
        end_date = current_date

        # Obtener los datos de la API de USGS
        url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
        parameters = {
            'format': 'geojson',
            'starttime': start_date.strftime('%Y-%m-%d'),
            'endtime': end_date.strftime('%Y-%m-%d'),
            'minmagnitude': '2.5',
            'minlatitude': '-5.266',
            'maxlatitude': '15.708',
            'minlongitude': '276.328',
            'maxlongitude': '293.906',
            'limit': 20000
        }
        response = requests.get(url, params=parameters)
        data = response.json()

        # Insertar los datos en la tabla
        table_ref = client.dataset(dataset_id).table(table_id)
        table = client.get_table(table_ref)

        rows = []
        for feature in data['features']:
            properties = feature['properties']
            time_str = properties['time']
            time = datetime.fromtimestamp(int(time_str) / 1000)  # Convertir a datetime
            row = (
                properties['ids'],
                properties['mag'],
                properties['place'],
                time,
                properties['url']
            )
            rows.append(row)

        client.insert_rows(table, rows)

    return 'Data actualizada correctamente'

y estos el contenido del requirements.txt

functions-framework==3.*
requests
google-cloud-bigquery